# Demo of SE-Gym
This is a demo of running LLM-Prompt-based agents in the SE-Gym environment.

In [ ]:
import se_gym
import importlib
import dotenv
import logging

importlib.reload(se_gym.api)

dotenv.load_dotenv("./se_gym/.env")

env = se_gym.api.make("dummy")

In [ ]:
MAX_TIME_STEPS = 20
logging.basicConfig(
    format="%(asctime)s %(levelname)s:%(message)s",
    level=logging.INFO,
    datefmt="%I:%M:%S",
    handlers=[logging.FileHandler("se_gym.log"), logging.StreamHandler()],
)
logging.getLogger("caller").setLevel(level=logging.DEBUG)
logging.getLogger("dockerconnector").setLevel(level=logging.DEBUG)
logging.getLogger("genetic").setLevel(level=logging.DEBUG)


In [ ]:
import se_gym.genetic

state = env.reset()

# Multiple initial prompts, as we are using a genetic algorithm
INITIAL_θ = [
    "You are a Software engineer. Suggest Code to fix the issue. Use the provided code snippet to understand the issue. Write tests to verify your fix.",
    "Fix the issue.",
    "The code is broken, as described in the provided code snippet. Fix it. Write tests to verify your fix.",
]


se_gym.config.MODEL_NAME = "llama3:8b"

# Add your client here
client = se_gym.openai_lmu.get_lmu_openai_client()

π = se_gym.Sampler(client, code_base_root=env.reset().path)

population = se_gym.genetic.Population(
    client=client,
    initial_individuals=INITIAL_θ,
    elite_size=0,  # No elitism
    mutation_rate=0.2,
    crossover_rate=0.7,
    sampler=π,
)

observer = se_gym.ManualObserver(
    files=[
        "./temp/kyrillschmidpythonenv/src/python_env/__main__.py",
        "./temp/kyrillschmidpythonenv/tests/my_test.py",
    ]
)

R = se_gym.fitness.num_failed_tests


In [ ]:
for iteration in range(1):
    r = 0
    s_t = env.reset()
    for t in range(MAX_TIME_STEPS):
        o_t = observer(s_t)  # observation at time t
        a_t = population.sample(o_t)  # actions at time t
        s_t = env.step(a_t)  # apply actions at time t to get next state
        current_r = [R(s_) for s_ in s_t]
        r += sum(current_r)
        print(f"Current reward: {current_r}")
        # evolve the population based on the current reward
        population.evolve(current_r) 
    ## evolve the population based on the total reward 
    # population.evolve(r) 
